In [1]:
import networks
import networks3d
import torchsummary
import torch
import trainer
import trainer3d
from torch import nn
%load_ext autoreload
%autoreload 2

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [19]:
enc = networks.StyleEncoder(1, 3, 2, 8,norm='in',activ='relu', pad_type='replicate').to(device)
# model = enc()
torchsummary.summary(enc,(3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
  ReplicationPad2d-1          [-1, 3, 134, 134]               0
            Conv2d-2          [-1, 2, 128, 128]             296
    InstanceNorm2d-3          [-1, 2, 128, 128]               0
              ReLU-4          [-1, 2, 128, 128]               0
       Conv2dBlock-5          [-1, 2, 128, 128]               0
  ReplicationPad2d-6          [-1, 2, 130, 130]               0
            Conv2d-7            [-1, 4, 64, 64]             132
    InstanceNorm2d-8            [-1, 4, 64, 64]               0
              ReLU-9            [-1, 4, 64, 64]               0
      Conv2dBlock-10            [-1, 4, 64, 64]               0
 ReplicationPad2d-11            [-1, 4, 66, 66]               0
           Conv2d-12            [-1, 8, 32, 32]             520
   InstanceNorm2d-13            [-1, 8, 32, 32]               0
             ReLU-14            [-1, 8,

In [21]:
mlp = networks.MLP(8,4,8,3).to(device)
torchsummary.summary(mlp,(8,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 8]              72
              ReLU-2                    [-1, 8]               0
       LinearBlock-3                    [-1, 8]               0
            Linear-4                    [-1, 8]              72
              ReLU-5                    [-1, 8]               0
       LinearBlock-6                    [-1, 8]               0
            Linear-7                    [-1, 4]              36
       LinearBlock-8                    [-1, 4]               0
Total params: 180
Trainable params: 180
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [27]:
params={
    'n_layer': 4,
    'gan_type': 'lsgan',
    'dim': 4,
    'norm': 'none',
    'activ': 'lrelu',
    'num_scales': 2,
    'pad_type': 'replicate'
}
        

In [22]:
dis = networks3d.MsImageDis(2, params).to(device)
torchsummary.summary(dis,(2,128,128,128))

replicate
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
  ReplicationPad3d-1     [-1, 2, 130, 130, 130]               0
            Conv3d-2        [-1, 4, 64, 64, 64]             516
         LeakyReLU-3        [-1, 4, 64, 64, 64]               0
       Conv3dBlock-4        [-1, 4, 64, 64, 64]               0
  ReplicationPad3d-5        [-1, 4, 66, 66, 66]               0
            Conv3d-6        [-1, 8, 32, 32, 32]           2,056
         LeakyReLU-7        [-1, 8, 32, 32, 32]               0
       Conv3dBlock-8        [-1, 8, 32, 32, 32]               0
  ReplicationPad3d-9        [-1, 8, 34, 34, 34]               0
           Conv3d-10       [-1, 16, 16, 16, 16]           8,208
        LeakyReLU-11       [-1, 16, 16, 16, 16]               0
      Conv3dBlock-12       [-1, 16, 16, 16, 16]               0
 ReplicationPad3d-13       [-1, 16, 18, 18, 18]               0
           Conv3d-14         

In [29]:
dis = networks.MsImageDis(2, params).to(device)
torchsummary.summary(dis,(2,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
  ReplicationPad2d-1          [-1, 2, 130, 130]               0
            Conv2d-2            [-1, 4, 64, 64]             132
         LeakyReLU-3            [-1, 4, 64, 64]               0
       Conv2dBlock-4            [-1, 4, 64, 64]               0
  ReplicationPad2d-5            [-1, 4, 66, 66]               0
            Conv2d-6            [-1, 8, 32, 32]             520
         LeakyReLU-7            [-1, 8, 32, 32]               0
       Conv2dBlock-8            [-1, 8, 32, 32]               0
  ReplicationPad2d-9            [-1, 8, 34, 34]               0
           Conv2d-10           [-1, 16, 16, 16]           2,064
        LeakyReLU-11           [-1, 16, 16, 16]               0
      Conv2dBlock-12           [-1, 16, 16, 16]               0
 ReplicationPad2d-13           [-1, 16, 18, 18]               0
           Conv2d-14             [-1, 3

In [23]:
hyperparameters={
"image_save_iter": 1000,         # How often do you want to save output images during training
"image_display_iter": 100,       # How often do you want to display output images during training
"display_size": 8,               # How many images do you want to display each time
"snapshot_save_iter": 10000,     # How often do you want to save trained models
"log_iter": 1,                   # How often do you want to log the training stats

# optimization options
"max_iter": 1000000 ,            # maximum number of training iterations
"batch_size": 1   ,              # batch size
"weight_decay": 0.0001  ,        # weight decay
"beta1": 0.5  ,                  # Adam parameter
"beta2": 0.999 ,                 # Adam parameter
"init": "kaiming" ,                # initialization [gaussian/kaiming/xavier/orthogonal]
"lr": 0.0001 ,                   # initial learning rate
"lr_policy": "step"    ,           # learning rate scheduler
"step_size": 100000 ,            # how often to decay learning rate
"gamma": 0.5    ,                # how much to decay learning rate
"gan_w": 1       ,               # weight of adversarial loss
"recon_x_w": 10  ,               # weight of image reconstruction loss
"recon_s_w": 1  ,                # weight of style reconstruction loss
"recon_c_w": 1 ,                 # weight of content reconstruction loss
"recon_x_cyc_w": 0 ,             # weight of explicit style augmented cycle consistency loss
"vgg_w": 0   ,                   # weight of domain-invariant perceptual loss

# model options
"gen":{
    
  "dim": 4  ,                   # number of filters in the bottommost layer
  "mlp_dim": 2  ,              # number of filters in MLP
  "style_dim": 4  ,              # length of style code
  "activ": "relu" ,                # activation function [relu/lrelu/prelu/selu/tanh]
  "n_downsample":3  ,           # number of downsampling layers in content encoder
  "n_res": 2         ,           # number of residual blocks in content encoder/decoder
  "pad_type": "replicate",      
    
},
       # padding type [zero/reflect]
"dis":{
  "dim": 4 ,                    # number of filters in the bottommost layer
  "norm": "none" ,                 # normalization layer [none/bn/in/ln]
  "activ": "lrelu" ,               # activation function [relu/lrelu/prelu/selu/tanh]
  "n_layer": 2   ,               # number of layers in D
  "gan_type": "lsgan" ,            # GAN loss [lsgan/nsgan]
  "num_scales": 2    ,           # number of scales
  "pad_type": "replicate"          # padding type [zero/reflect]
},
 

# data options
"input_dim_a": 3,                              # number of image channels [1/3]
"input_dim_b": 3   ,                           # number of image channels [1/3]
"num_workers": 0   ,                           # number of data loading threads
"new_size": 24    ,                           # first resize the shortest image side to this size
"crop_image_height": 24  ,                    # random crop image of this height
"crop_image_width": 24      ,                 # random crop image of this width
"data_root": './datasets/demo_edges2handbags/'     # dataset folder location
}

In [26]:
munit_trainer = trainer3d.MUNIT_Trainer(hyperparameters).to(device)
torchsummary.summary(munit_trainer,[(3,32,32,32),(3,32,32,32)])

torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])


RuntimeError: weight should contain 64 elements not 256

In [17]:
a = torch.tensor([16])
a.repeat(a)

tensor([16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16])

In [ ]:
self.running_mean.repeat(b)